Copyright **`(c)`** 2022 Giovanni Squillero `<squillero@polito.it>`  
[`https://github.com/squillero/computational-intelligence`](https://github.com/squillero/computational-intelligence)  
Free for personal or classroom use; see [`LICENSE.md`](https://github.com/squillero/computational-intelligence/blob/master/LICENSE.md) for details.  


# Lab 3: ES

## Task

Write agents able to play [*Nim*](https://en.wikipedia.org/wiki/Nim), with an arbitrary number of rows and an upper bound $k$ on the number of objects that can be removed in a turn (a.k.a., *subtraction game*).

The goal of the game is to **avoid** taking the last object.

* Task2.1: An agent using fixed rules based on *nim-sum* (i.e., an *expert system*)
* Task2.2: An agent using evolved rules using ES

## Instructions

* Create the directory `lab2` inside the course repo 
* Put a `README.md` and your solution (all the files, code and auxiliary data if needed)

## Notes

* Working in group is not only allowed, but recommended (see: [Ubuntu](https://en.wikipedia.org/wiki/Ubuntu_philosophy) and [Cooperative Learning](https://files.eric.ed.gov/fulltext/EJ1096789.pdf)). Collaborations must be explicitly declared in the `README.md`.
* [Yanking](https://www.emacswiki.org/emacs/KillingAndYanking) from the internet is allowed, but sources must be explicitly declared in the `README.md`.



In [147]:
import logging
from pprint import pprint, pformat
from collections import namedtuple
import random
from copy import deepcopy
import numpy as np


## The *Nim* and *Nimply* classes

In [148]:
Nimply = namedtuple("Nimply", "row, num_objects")


In [149]:
class Nim:
    def __init__(self, num_rows: int, k: int = None) -> None:
        self._rows = [i * 2 + 1 for i in range(num_rows)]
        self._k = k

    def __bool__(self): #verifica se ci sono oggetti in qualche riga
        return sum(self._rows) > 0

    def __str__(self): #stringa per stato attuale
        return "<" + " ".join(str(_) for _ in self._rows) + ">"

    @property 
    def rows(self) -> tuple: #restituisce una tupla con il numero corrente di oggwti in ogni riga
        return tuple(self._rows)

    #viene utilizzato per effettuare una mossa nel gioco specificando una riga e il numero di oggetti da rimuovere
    def nimming(self, ply: Nimply) -> None:
        row, num_objects = ply
        assert self._rows[row] >= num_objects
        assert self._k is None or num_objects <= self._k
        self._rows[row] -= num_objects


## Sample (and silly) startegies 

In [150]:
#mossa casuale

def pure_random(state: Nim) -> Nimply:
    """A completely random move"""
    row = random.choice([r for r, c in enumerate(state.rows) if c > 0])
    num_objects = random.randint(1, state.rows[row])
    return Nimply(row, num_objects)


In [151]:
#massimo possibile nella rig più bassa

def gabriele(state: Nim) -> Nimply:
    """Pick always the maximum possible number of the lowest row"""
    possible_moves = [(r, o) for r, c in enumerate(state.rows) for o in range(1, c + 1)]
    return Nimply(*max(possible_moves, key=lambda m: (-m[0], m[1])))


In [152]:
def adaptive(state: Nim) -> Nimply:
    """A strategy that can adapt its parameters"""
    genome = {"love_small": 0.5}
    
    # Estrai il parametro "love_small" dal genoma
    love_small_param = genome["love_small"]
    
    # Calcola alcuni valori basati sullo stato attuale del gioco 
    # Conta il numero di righe attive, cioè il numero di righe che contengono ancora oggetti. 
    active_rows_number = sum(o > 0 for o in state.rows)
    # Trova la riga più corta, cioè quella con il minor numero di oggetti. 
    # La lambda function viene utilizzata come chiave per determinare il MINIMO in base al numero di oggetti.
    shortest_row = min((x for x in enumerate(state.rows) if x[1] > 0), key=lambda y: y[1])[0]
    # Trova la riga più lunga, cioè quella con il maggior numero di oggetti.
    # La lambda function viene utilizzata come chiave per determinare il MASSIMO in base al numero di oggetti.
    longest_row = max((x for x in enumerate(state.rows)), key=lambda y: y[1])[0]
    
    # Prendi una decisione basata sul parametro "love_small" 
    if love_small_param > 0.5:
        # Se "love_small" è maggiore di 0.5, scegli la mossa più piccola sulla fila più corta
        ply = Nimply(active_rows_number, 1) 
    else:
        # Altrimenti, scegli la mossa più grande sulla fila più lunga
        ply = Nimply(longest_row, state.rows[longest_row])
    
    return ply

#nvece di calcolare esplicitamente questi valori, la strategia seleziona casualmente una fila non vuota
# e effettua una mossa in base alla fila scelta. 
# Ciò semplifica la strategia pur incorporando ancora il parametro adattivo.

def adaptive_alternativa(state: Nim) -> Nimply:
    genome = {"love_small": 0.5}
    
    # Estrai il parametro "love_small" dal genoma
    love_small_param = genome["love_small"]
    
    # Calcola alcuni valori basati sullo stato attuale del gioco
    active_rows = [i for i, oggetti in enumerate(state.rows) if oggetti > 0]
    
    # Prendi una decisione basata sul parametro e lo stato del gioco
    if love_small_param > 0.5:
        # Scegli la mossa più piccola su una fila non vuota a caso
        fila_scelta = random.choice(active_rows)
        ply = Nimply(fila_scelta, 1)
    else:
        # Scegli la mossa più grande su una fila non vuota a caso
        fila_scelta = random.choice(active_rows)
        ply = Nimply(fila_scelta, state.rows[fila_scelta])
    
    return ply

def adaptive_dynamic(state: Nim) -> Nimply:
    genome = {"love_small": 0.5}
    
    # Estrai il parametro "love_small" dal genoma
    love_small_param = genome["love_small"]
    
    # Calcola alcuni valori basati sullo stato attuale del gioco
    active_rows = [i for i, oggetti in enumerate(state.rows) if oggetti > 0]
    
    # Rendi i parametri adattivi più dinamici  in base alla percentuale di righe vuote rispetto al totale delle righe del gioco.
    #n modo che sia proporzionale alla quantità di spazio disponibile nel gioco. 
    # In sostanza, se ci sono più righe vuote, love_small_param sarà influenzato di più, e viceversa.
    love_small_param *= state.rows.count(0) / len(state.rows)
    
    # Prendi una decisione basata sul parametro e lo stato del gioco
    if love_small_param > 0.5:
        # Scegli la mossa più piccola su una fila non vuota a caso
        fila_scelta = random.choice(active_rows)
        ply = Nimply(fila_scelta, 1)
    else:
        # Scegli la mossa più grande su una fila non vuota a caso
        fila_scelta = random.choice(active_rows)
        ply = Nimply(fila_scelta, state.rows[fila_scelta])
    
    return ply





In [153]:
import numpy as np


def nim_sum(state: Nim) -> int:
    tmp = np.array([tuple(int(x) for x in f"{c:032b}") for c in state.rows])
    xor = tmp.sum(axis=0) % 2
    return int("".join(str(_) for _ in xor), base=2)

#analizza lo stato attuale e restituisce le possibili modde e il risultato finale somma dopo ogni mossa
def analize(raw: Nim) -> dict:
    cooked = dict()
    cooked["possible_moves"] = dict()
    for ply in (Nimply(r, o) for r, c in enumerate(raw.rows) for o in range(1, c + 1)):
        tmp = deepcopy(raw)
        tmp.nimming(ply)
        cooked["possible_moves"][ply] = nim_sum(tmp)
    return cooked

#ricerca di mossa migliori in base alle molsse possibili
def optimal(state: Nim) -> Nimply:
    analysis = analize(state)
    logging.debug(f"analysis:\n{pformat(analysis)}")
    spicy_moves = [ply for ply, ns in analysis["possible_moves"].items() if ns != 0]
    if not spicy_moves:
        spicy_moves = list(analysis["possible_moves"].keys())
    ply = random.choice(spicy_moves)
    return ply


## Oversimplified match

In [154]:
logging.getLogger().setLevel(logging.INFO)
#sialterna tra le strategie in base a quelle scelte dai giocatori
strategy = (adaptive_dynamic, pure_random)

nim = Nim(7)
logging.info(f"init : {nim}")
player = 0
while nim:
    ply = strategy[player](nim)
    logging.info(f"ply: player {player} plays {ply}")
    nim.nimming(ply)
    logging.info(f"status: {nim}")
    player = 1 - player
logging.info(f"status: Player {player} won!")


INFO:root:init : <1 3 5 7 9 11 13>
INFO:root:ply: player 0 plays Nimply(row=2, num_objects=5)
INFO:root:status: <1 3 0 7 9 11 13>
INFO:root:ply: player 1 plays Nimply(row=6, num_objects=13)
INFO:root:status: <1 3 0 7 9 11 0>
INFO:root:ply: player 0 plays Nimply(row=5, num_objects=11)
INFO:root:status: <1 3 0 7 9 0 0>
INFO:root:ply: player 1 plays Nimply(row=4, num_objects=9)
INFO:root:status: <1 3 0 7 0 0 0>
INFO:root:ply: player 0 plays Nimply(row=3, num_objects=7)
INFO:root:status: <1 3 0 0 0 0 0>
INFO:root:ply: player 1 plays Nimply(row=1, num_objects=3)
INFO:root:status: <1 0 0 0 0 0 0>
INFO:root:ply: player 0 plays Nimply(row=0, num_objects=1)
INFO:root:status: <0 0 0 0 0 0 0>
INFO:root:status: Player 1 won!
